In [1]:
import pandas as pd
import numpy as np
import os

import itertools

In [2]:
COVID_DIR = 'COVID_19'
CARD_FILENAME = 'card_20200717.csv'
CATEGORISING_FILENAME = '데이콘분류.xlsx'

## 선분류작업

In [3]:
standard_df = pd.read_excel(CATEGORISING_FILENAME).iloc[:14, :].T.reset_index()
standard_df.head()

,index,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,대중교통,버스,지하철,택시,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,교통,국산신차,렌트카,전기차충전소,주차장,주유소,자동차시트/타이어,카인테리어,중고자동차,이륜차판매,수입자동차,NaN,NaN,NaN,NaN
2,의류,캐쥬얼의류,정장,기타의류,신발,스포츠의류,내의판매점,맞춤복점,와이셔츠/타이,NaN,NaN,NaN,NaN,NaN,NaN
3,온라인,인터넷Mall,인터넷P/G,인터넷종합Mall,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,음식점,제과점,중국음식,서양읍식,스넥,일반한식,일식회집,한정식,기타건강식,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
division_1st = standard_df.iloc[:, 0]
division_1st

0     대중교통
1       교통
2       의류
3      온라인
4      음식점
5     실내활동
6       관광
7       의료
8     유흥주점
9     실외활동
10      교육
11      농업
Name: index, dtype: object

In [5]:
division_2nd = standard_df.iloc[:, 1:].values
division_2nd = list(map(lambda x: [w for w in x if str(w) != 'nan'], division_2nd))
div2list= list(itertools.chain(*division_2nd))

In [6]:
div2dict = {lmn: key for key, lmns in zip(division_1st, division_2nd) for lmn in lmns}

# 카드 소비 내역

In [7]:
card_df = pd.read_csv(CARD_FILENAME)
card_df['mrhst_induty_cl_nm'] = card_df['mrhst_induty_cl_nm'].str.replace(pat = ' ', repl = '')
card_df.shape

(3713952, 7)

In [8]:
object_df = card_df[card_df['mrhst_induty_cl_nm'].isin(div2list)]
object_df.shape

(1348866, 7)

In [9]:
object_df['대분류'] = object_df['mrhst_induty_cl_nm'].replace(div2dict)

/home/eunbinpark/miniconda3/envs/da/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [10]:
object_df = object_df[['receipt_dttm', '대분류', 'mrhst_induty_cl_nm', 'selng_cascnt']]
object_df.columns = ['datetime', '대분류', '중분류', 'sales count']

In [11]:
object_df = object_df.reset_index(drop = True)

In [16]:
def make_datetime(x):
    x = str(x)
    yr = x[:4]
    month = x[4:6]
    day = x[6:]
    date = f'{yr}-{month}-{day}'
    return date

In [17]:
object_df['datetime'] = object_df['datetime'].map(make_datetime)

## 코로나 확진자

In [18]:
covid_df = pd.read_csv(os.path.join(COVID_DIR, 'Time.csv'))
covid_df.shape

(163, 7)

In [19]:
covid_df = covid_df[['date', 'confirmed']]

In [20]:
covid_df.columns = ['datetime', 'covid_confirmed']

# Final Result

In [24]:
result_df = pd.merge(object_df, covid_df, on = 'datetime').sort_values('datetime').reset_index(drop = True)
result_df

,datetime,대분류,중분류,sales count,covid_confirmed
0,2020-01-20,음식점,제과점,50,1
1,2020-01-20,음식점,스넥,369,1
2,2020-01-20,음식점,중국음식,97,1
3,2020-01-20,음식점,일식회집,19,1
4,2020-01-20,음식점,일반한식,814,1
...,...,...,...,...,...
1290987,2020-06-14,의료,치과의원,8,12085
1290988,2020-06-14,음식점,제과점,179,12085
1290989,2020-06-14,의류,기타의류,23,12085
1290990,2020-06-14,음식점,제과점,145,12085


In [25]:
result_df.to_csv('internal_df.csv', index = False)